In [3]:
%matplotlib inline
import sys
import pandas as pd
import math
import matplotlib.pylab as plt
import matplotlib as mpl
from collections import Counter
import numpy as np
import subprocess
import re
import itertools
import mmh3
import json
import pickle
from datetime import datetime as dt
from datetime import time
from random import randrange, gauss, shuffle

In [4]:
# ----------------- #
## Handle raw data ##
# ----------------- #

def load_raw_month_data():
    df_short = pd.read_csv('../Data/raw_data/short.csv', sep=" ").loc[:,['#','user1','user2','timestamp']]
    df_all = pd.read_csv('../Data/raw_data/all.csv', sep=" ").loc[:,['#','user1','user2','timestamp']]

    # Fix bad column name assignment
    df_all.columns = ['user1','user2','timestamp','duration']
    df_short.columns = ['user1','user2','timestamp','duration']
    
    return df_short, df_all


def time_bin_network(df, window=3600):
    """Return iterator to partition network in 'window'-sized bins.
    
    Parameters
    ----------
    df : network dataframe
        Pandas dataframe with network loaded straight from csv.
    window : bin-size integer
        Size of network bins in seconds. This number controls
        the amount of bins.
        
    Returns
    -------
    df_slice : iterator
        Iterator object that yields one bin at a time.
    """
    
    try:
        assert window >= 300
        window += 0.01
    except AssertionError:
        raise AssertionError("""'window' must be greater than, \
or equal to 300.""")
        
    # Get timestamp column and extreme values
    col_t = df["timestamp"]
    min_t = min(col_t)
    max_t = max(col_t)    
    
    # Get timespan and number of network splits
    delta_time = max_t - min_t
    
    n_splits = int(math.ceil(delta_time/float(window)))

    for i in range(n_splits):
        lower_bound = df["timestamp"] > min_t + i*window
        upper_bound = df["timestamp"] < min_t + (i+1)*window
        df_slice = df[lower_bound][upper_bound]
        yield df_slice
        
        
def dump_binned_network(df, binsize, filename):
    """Store binned network into local file
    
    Parameters
    ----------
    df : network dataframe
        Pandas dataframe with network
    binsize : int
        Size of bins in resulting network
    filename : str
        Name of resulting file
    """
    
    binned_network = list(time_bin_network(df, window=binsize))
    
    with open('../Data/processed_data/binned_networks/'+filename+'.pickle', 'w') as outfile:
        pickle.dump(binned_network, outfile)
        
        
def create_loadable_bins():
    """Store a series of binned networks locally
    """
    
    bins = [300, 900, 1800, 3600, 86400, 604800]
    bin_names = ['5mins', '15mins', '30mins', 'hourly', 'daily', 'weekly']
    dataframes = [df_short, df_all]
    dataframe_names = ['_short', '_all']

    for df_, dfn_ in zip(dataframes,dataframe_names):
        for b,n in zip(bins,bin_names):
            print dfn_,n
            dump_binned_network(df_,b,n+dfn_)
            
def load_binned_network(kind,filename):
    with open('../Data/processed_data/binned_networks/'+kind+'/'+filename+'.pickle', 'r') as infile:
        return pickle.load(infile)

In [5]:
# ----------------------- #
## Reformat layered data ##
# ----------------------- #

def network_reformat_multiplex(layers, halflife=-1):
    """Return multiplex representation of multiplex network
    
    Parameters
    ----------
    halflife : number
        Halflife in seconds of relax-rate decay between layers.
        Defaults to -1.
    layers : pandas df formatted layers
        
    Returns
    -------
    net_file : string
        A network string in multiplex format
    int_to_hash : dict
        Key-value pairs of node integer id and original hash id
    """
    
    # Infomap will only work with node ids as indices.
    
    # Get all node ids in original md5 hash values
    nodes = set()
    for l, df in enumerate(layers):
        layer_nodes = set()
        layer_nodes.update(df["user1"])
        layer_nodes.update(df["user2"])
        nodes.update(layer_nodes)
        
    
    ##########################
    ## Add vertices to file ##
    ##########################
    
    out_file = "*Vertices %d" % len(nodes)
    
    # Node name book-keeping, and adding to file
    hashid_to_intid = {}
    intid_to_hashid = {}
    for i,n in enumerate(nodes):
        intid = i+1
        hashid = str(n)
        out_file += '\n%d "Node %s" 1.0' % (intid,hashid)
        hashid_to_intid[hashid] = intid
        intid_to_hashid[intid] = hashid

        
    #############################
    ## Add Intra-edges to file ##
    #############################
    
    out_file += "\n*Multiplex\n# Intra edges: layer node layer node weight"
    
    for l, df in enumerate(layers):
        user1 = df["user1"]
        user2 = df["user2"]
            
        edges = zip(user1, user2)
        
        # Add weights. REDUNDANT FOR 5MINS TIMESLICES BECAUSE PPL ONLY MEET ONCE HERE.
        edges = [(e[0],e[1],w) for e,w in Counter(edges).items()]
        
        # Add Intra-edges to file
        for i,j,w in edges:
            out_file += '\n%d %s %d %s %d' % (l+1,hashid_to_intid[i], l+1,hashid_to_intid[j],w) #+1 because 1 is first layer index
            
    
    #############################
    ## Add Inter-edges to file ##
    #############################
    
    out_file += "# Intra edges: layer node layer node weight"
    
    # Infinte halflife (represented as -1)
    if halflife == -1:
        return out_file
    
    # Relax decay function
    def N(t):
        tau = halflife/np.log(2)
        return np.exp(-t/float(tau))
                        
    for l1, df1 in enumerate(layers):
        nodes1 = set(list(df1['user1'].values)+list(df1['user2'].values))
        for l2, df2 in enumerate(layers):    
            if not l2 > l1:
                continue   
            
            nodes2 = set(list(df2['user1'].values)+list(df2['user2'].values))
            common_nodes = nodes1 & nodes2
            time_diff = df2['timestamp'].values[0] - df1['timestamp'].values[0]
            
            for n in common_nodes:
                out_file += '\n%d %s %d %s %f' % (l1+1,hashid_to_intid[n],l2+1,hashid_to_intid[n],N(time_diff))
    
    return out_file, intid_to_hashid

In [6]:
# ----------------------- #
## Run Infomap algorithm ##
# ----------------------- #

def community_detection_multiplex(layers, halflife=2400):
    """Run multiplex community detection because Python implementation has no docs
    
    Parameters
    ----------
    halflife : number
        Halflife in seconds of relax-rate decay between layers.
        Defaults to -1.
    layers : layers (pandas dfs)
    
    Returns
    -------
    communities : list of lists
    """
    
    def parse_communities():
        with open('output/'+multiplex_network_filename+"_expanded.clu", 'r') as infile:
            multiplex_network_clusters = infile.read()

        # Get layers, nodes and clusters from _extended.clu file
        la_no_clu = re.findall(r'\d+ \d+ \d+ \d\.\d+', multiplex_network_clusters) # ["30 1 2 0.00800543",...]
        la_no_clu = [tuple(i.split()) for i in la_no_clu]

        communities_json = {}
        for layer, node, cluster, _ in la_no_clu:
            try:
                communities_json[int(layer)].add((intid_to_hashid[int(node)], int(cluster)))
            except KeyError:
                # Will run once for every layer
                communities_json.update({int(layer): {(intid_to_hashid[int(node)], int(cluster))}})

        return communities_json

    
    # Get network in mutliplex string format and define filename
    multiplex_network, intid_to_hashid = network_reformat_multiplex(layers, halflife=halflife)
    multiplex_network_filename = 'multiplex-network'

    # Store locally
    with open("input/"+multiplex_network_filename+".net", 'w') as outfile:
        outfile.write(multiplex_network)
    
    # Run Infomap for multiplex network
    subprocess.call(['./Infomap/Infomap', 'input/'+multiplex_network_filename+".net", 
                     'output/', '-i', 'multiplex', '--overlapping', '--map', '--clu', '--tree', '--expanded'])
    
    parsed_communities = parse_communities()
    
    hash_clu = [item for sublist in parsed_communities.values() for item in sublist]
    communities = dict()
    for key, group in itertools.groupby(hash_clu, lambda x: x[1]):
        for thing in group:
            try:
                communities[key].append(thing[0])
            except KeyError:
                communities[thing[1]] = [thing[0]]
    communities = dict((k,set(v)) for k,v in communities.items())

    d3_ready = {}
    for layer, group in parsed_communities.items():
        layer_communities = {}
        for no, clu in group:
            try:
                layer_communities[clu].append(no)
            except KeyError:
                layer_communities[clu] = [no]
        d3_ready[layer] = layer_communities

    return communities, d3_ready

In [7]:
# -------------------------------- #
## Compute community similarities ##
# -------------------------------- #

def get_similarity(i,j,communities, count=False):
    """communities : Communities in each layer"""
    sim_counter = len(communities[i] & communities[j])
    tot_counter = len(communities[i] | communities[j])
    
    if count:
        return sim_counter / float(tot_counter), sim_counter
    else:
        return sim_counter / float(tot_counter)

def compute_similarity_matrix(communities):
    dim = len(communities)
    X = {}
    for i in range(dim):
        for j in range(dim):
            sim, count = get_similarity(i+1,j+1,communities, count=True)
            try:
                X['c'+str(i+1)].update({'c'+str(j+1): {'sim': sim, 'count': count}})
            except KeyError:
                X['c'+str(i+1)] = {'c'+str(j+1): {'sim': sim, 'count': count}}

    return X

In [8]:
## ------------------- ##
## Load binned network ##
## ------------------- ##

#month_all_5mins = load_binned_network('1month_data','5mins_all')
month_sho_5mins = load_binned_network('1month_data','5mins_short')
#month_all_15mins = load_binned_network('1month_data','15mins_all')
#month_sho_15mins = load_binned_network('1month_data','15mins_short')
#month_all_30mins = load_binned_network('1month_data','30mins_all')
#month_sho_30mins = load_binned_network('1month_data','30mins_short')
#month_all_hourly = load_binned_network('1month_data','hourly_all')
##month_sho_hourly = load_binned_network('1month_data','hourly_short')
#month_all_daily = load_binned_network('1month_data','daily_all')
#month_sho_daily = load_binned_network('1month_data','daily_short')
#month_all_weekly = load_binned_network('1month_data','weekly_all')
#month_sho_weekly = load_binned_network('1month_data','weekly_short')

In [9]:
## ------------------------ ##
## Define LAYERS to analyse ##
## ------------------------ ##
layers = month_sho_5mins[576:864]

## ------------------- ##
## Compute communities ##
## ------------------- ##
communities, layer_communities = community_detection_multiplex(layers, halflife=300)
similarities = compute_similarity_matrix(communities)

### Build dataset for d3

In [80]:
# Canvas parameters
actual_width = len(set([item for sublist in communities.values() for item in sublist])) #number of nodes
actual_height = len(layers)

min_group_size = 1

In [81]:
def is_valid_location(new_block, existing_blocks):
    """Check whether random horizontal location for block is unoccupied"""
    pad = 5
    nb_x = new_block['x']
    nb_w = new_block['w']
    nb_range = set(range(int(nb_x-pad),int(nb_x+nb_w+pad)))
    for block in existing_blocks:
        eb_x = block['x']
        eb_w = block['w']
        eb_range = set(range(int(eb_x-pad),int(eb_x+eb_w+pad)))
        if len(nb_range & eb_range) != 0:
            return False

    return True

# Set working width and height
width = actual_width*2
height = actual_height


#------#
# Time #
#------#

# Start, termination, timestep variables
t0 = str(dt.fromtimestamp(layers[0]['timestamp'].values[0])) # 2014-02-03 00:05:00
tt = str(dt.fromtimestamp(layers[-1]['timestamp'].values[-1])) # 2014-02-04 00:00:00
d_t = str((layers[1]['timestamp'].values[0]-layers[0]['timestamp'].values[0])/60) # 5 (minutes)

thickness = layers[1]['timestamp'].values[0] - \
            layers[0]['timestamp'].values[0] # 300 (seconds)

# Lines marking important points in time
grid_times = [time(h) for h in [8,12,13,17]]
grid_ticks = dict((i+1,str(dt.fromtimestamp(l['timestamp'].values[-1]))) 
                   for i, l in enumerate(layers) 
                   if dt.fromtimestamp(l['timestamp'].values[-1]).time() in grid_times)

# Time tick labels
label_times = [time(h) for h in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]]
label_ticks = dict((i+1,str(dt.fromtimestamp(l['timestamp'].values[-1]))) 
                  for i, l in enumerate(layers) 
                  if dt.fromtimestamp(l['timestamp'].values[-1]).time() in label_times)


#----------------------#
# Build data structure #
#----------------------#

# Initiate data structure
ds = {
    'meta': {'w': width, 'h': height},
    'time': {'t0': t0, 
             'tt': tt,
             'dt': d_t,
             'ticks': {'label_ticks': label_ticks,
                       'grid_ticks': grid_ticks}
            },
    'sims': similarities,
    'coms': {},
    'layer_networks': {}
}

# Compute geometry and location of community blocks
preferred_locations = {}
prev_layer_blocks = {}
for l,coms in layer_communities.items():
    # Community-size pairs
    com_size_pairs = [(k, len(v)) for k,v in coms.items() if len(v) >= min_group_size]
    
    # Compute blocks also in previous layer (EXISTING BLOCKS)
    this_layer_blocks = {}
    not_in_prev = []
    for c,s in com_size_pairs:
        
        com = "c"+str(c)
        if com in prev_layer_blocks:
            prev_block = prev_layer_blocks[com]
            this_x = (prev_block['w']-s)/2.0+prev_block['x']
            block = {'x':this_x, 'w': s}
            this_layer_blocks[com] = block
            
            points = [[this_x,l-1], [this_x+s,l-1], [this_x,l], [this_x+s,l]]
            for p in points:
                ds['coms'][com]['blocks'][-1]['points'].insert(
                    len(ds['coms'][com]['blocks'][-1]['points'])/2, p
                )
                
                ds['coms'][com]['tt'] = layers[l-1]['timestamp'].values[-1]
                
                ds['coms'][com]['duration'] = \
                    ds['coms'][com]['tt'] - \
                    ds['coms'][com]['t0']
                
                ds['coms'][com]['avg_size'] += s
                ds['coms'][com]['avg_size'] /= 2.0
                if s > ds['coms'][com]['max_size']:
                    ds['coms'][com]['max_size'] = s
                elif s < ds['coms'][com]['min_size']:
                    ds['coms'][com]['min_size'] = s
        else:
            not_in_prev.append((c,s))

    # Compute blocks not in previous layer (NEW BLOCKS)
    existing_blocks = this_layer_blocks.values()
    for c, s in not_in_prev:
        com = "c"+str(c)
        
        # First try to put the block at its preferred location
        if com in preferred_locations:
            
            this_x = preferred_locations[com]-s/2.0
            block = {'x': this_x, 'w': s}
            
            if is_valid_location(block, existing_blocks):
                
                this_layer_blocks[com] = block
                existing_blocks = this_layer_blocks.values()
                
                points = [[this_x,l-1], [this_x+s,l-1], [this_x,l], [this_x+s,l]]
                for i, p in enumerate(points):
                    if com not in ds['coms']:
                        ds['coms'][com] = {'blocks': [], 
                                           't0': layers[l-1]['timestamp'].values[0], 
                                           'tt': layers[l-1]['timestamp'].values[-1],
                                           'duration': thickness,
                                           'abs_size': len(communities[c]),
                                           'min_size': s,
                                           'max_size': s,
                                           'avg_size': s}
                    if i == 0:
                        ds['coms'][com]['blocks'].append({'points': [p], 'c': com})
                    else:
                        ds['coms'][com]['blocks'][-1]['points'].insert(
                            len(ds['coms'][com]['blocks'][-1]['points'])/2, p
                        )
                        ds['coms'][com]['avg_size'] += s
                        ds['coms'][com]['avg_size'] /= 2.0
                        if s > ds['coms'][com]['max_size']:
                            ds['coms'][com]['max_size'] = s
                        elif s < ds['coms'][com]['min_size']:
                            ds['coms'][com]['min_size'] = s
                
                # SUCCES! Continue to next community
                continue
                
        # If that fails, go ahead and place the block wherever it fits!
        counter = 0
        while True:
            counter += 1
            
            if counter > 100000:
                print "Can't place block anywhere"
                print l
                sys.exit()
                
            this_x = randrange(0,width-s)
            block = {'x':this_x, 'w': s}
            if is_valid_location(block, existing_blocks):
                this_layer_blocks[com] = block
                existing_blocks = this_layer_blocks.values()
                
                points = [[this_x,l-1], [this_x+s,l-1], [this_x,l], [this_x+s,l]]
                for i, p in enumerate(points):
                    if com not in ds['coms']:
                        ds['coms'][com] = {'blocks': [], 
                                           't0': layers[l-1]['timestamp'].values[0], 
                                           'tt': layers[l-1]['timestamp'].values[-1],
                                           'duration': thickness,
                                           'abs_size': len(communities[c]),
                                           'min_size': s,
                                           'max_size': s,
                                           'avg_size': s}
                    if i == 0:
                        ds['coms'][com]['blocks'].append({'points': [p], 'c': com})
                    else:
                        ds['coms'][com]['blocks'][-1]['points'].insert(
                            len(ds['coms'][com]['blocks'][-1]['points'])/2, p
                        )
                        ds['coms'][com]['avg_size'] += s
                        ds['coms'][com]['avg_size'] /= 2.0
                        if s > ds['coms'][com]['max_size']:
                            ds['coms'][com]['max_size'] = s
                        elif s < ds['coms'][com]['min_size']:
                            ds['coms'][com]['min_size'] = s
                break
            
        preferred_locations[com] = block['x']+block['w']/2.0
        
        
    prev_layer_blocks = this_layer_blocks
    
# Add networks
def layer_networks(filename, layer_communities):
    
    with open(filename, 'r') as infile:
        rawstring = infile.read()
        
    rawstring_nodes, rawstring_edges = rawstring.split("#")[:-1]
    
    nodes_map = dict((int(re.findall(r'\d+', n)[0]), re.findall(r'[0-9a-f]{30}', n)[0]) for n in 
                      re.findall(r'\d+ "Node [0-9a-f]{30}" \d\.\d', rawstring_nodes))
    nodes_map_reverse = dict((v,k) for k,v in nodes_map.items())
    
    ln = {'data': {}}
    
    e_li = [e_str.split() for e_str in rawstring_edges.split('\n')[1:]] # [['1', '347', '1', '348', '1'],...]
    
    # Add edges
    for e in e_li:
        layer, source, target, value = int(e[0]), int(e[1]), int(e[3]), int(e[4])
        group_source = [g for g,n in layer_communities[layer].items() if nodes_map[source] in n][0]
        group_target = [g for g,n in layer_communities[layer].items() if nodes_map[target] in n][0]
        if group_source == group_target:
            divisor = 10.0
        else:
            divisor = 10.0
            
        if "c"+str(group_source) not in com_cols or "c"+str(group_target) not in com_cols:
            continue
    
        edge = {'source': source, 'target': target, 'value': value/divisor}
        try:
            ln['data'][layer]['links'].append(edge)
        except KeyError:
            ln['data'][layer] = {'links': [edge]}
        
        for n1, n2 in [(source, target),(target, source)]:
            try:
                ln['data'][layer]['links_dict'][n1].append(n2)
            except KeyError:
                try:
                    ln['data'][layer]['links_dict'][n1] = [n2]
                except KeyError:
                    ln['data'][layer]['links_dict'] = {n1: [n2]}
                
            
    # Add nodes
    for layer, edges_and_nodes in ln['data'].items():
        edges = edges_and_nodes['links']
        nodes_names = set()
        for e in edges:
            nodes_names.add(e['source'])
            nodes_names.add(e['target'])
        nodes = []
        for nn in nodes_names:
            group = [g for g,n in layer_communities[layer].items()
                         if nodes_map[nn] in n][0]
            try:
                col = com_cols['c'+str(group)]
            except KeyError:
                #col = 'rgb(%d,%d,%d)' % (200,200,200)
                continue
            node = {'name': nn, 'id': nodes_map[nn], 'group': group}
            try:
                ln['data'][layer]['nodes'][nn] = node
            except KeyError:
                ln['data'][layer].update({'nodes': {nn: node}})
                
            try:
                ln['data'][layer]['groups'][group].append(node['name'])
            except KeyError:
                try:
                    ln['data'][layer]['groups'][group] = [node['name']]
                except KeyError:
                    ln['data'][layer]['groups'] = {group: [node['name']]}
    
    return ln

layer_networks = layer_networks('input/multiplex-network.net', layer_communities)
ds['layer_networks'] = layer_networks

In [82]:
## ----------------------------- ##
## Store D3-ready data structure ##
## ----------------------------- ##

with open('Visualisation/data/dataset1.json', 'w') as outfile:
    json.dump(ds,outfile)

In [79]:
ds['coms']['c1']

{'abs_size': 15,
 'avg_size': 5.195083607700273,
 'blocks': [{'c': 'c1',
   'points': [[213, 94],
    [213, 95],
    [214.5, 95],
    [214.5, 96],
    [216.0, 96],
    [216.0, 97],
    [214.0, 97],
    [214.0, 98],
    [216.5, 98],
    [216.5, 99],
    [216.5, 99],
    [216.5, 100],
    [216.5, 100],
    [216.5, 101],
    [217.0, 101],
    [217.0, 102],
    [216.5, 102],
    [216.5, 103],
    [216.5, 103],
    [216.5, 104],
    [217.0, 104],
    [217.0, 105],
    [216.5, 105],
    [216.5, 106],
    [216.0, 106],
    [216.0, 107],
    [216.5, 107],
    [216.5, 108],
    [217.0, 108],
    [217.0, 109],
    [215.5, 109],
    [215.5, 110],
    [216.0, 110],
    [216.0, 111],
    [217.0, 111],
    [217.0, 112],
    [216.0, 112],
    [216.0, 113],
    [216.5, 113],
    [216.5, 114],
    [217.0, 114],
    [217.0, 115],
    [217.0, 115],
    [217.0, 116],
    [215.5, 116],
    [215.5, 117],
    [216.0, 117],
    [216.0, 118],
    [217.0, 118],
    [217.0, 119],
    [215.5, 119],
    [215.5, 12